In [1]:
from IPython.display import display, HTML
import os
import psycopg2
import json
import pandas as pd


conn = psycopg2.connect(
    host="localhost",
    database="tpch",
    user="postgres",
    password="qwer1234")
print(conn)
cur = conn.cursor()

<connection object at 0x7fd65c835800; dsn: 'user=postgres password=xxx dbname=tpch host=localhost', closed: 0>


In [2]:
#We converting json execution plan from postgres to indexed list of all nodes + marking their parent nodes
#using Breadth First Search algorithm
def bfs(node, node_id):
    visited = []
    queue = []
    node['parent'] = None
    queue.append(node)
    while queue:
        m = queue.pop(0)
        
        #Proceed possible filter and group_by field via creating extra node
        #We assume, that if both of them are present, filtering is taking place after grouping operation
        #(Filter --> GroupBy equals Filter after GroupBy, because of tree(child node --> parent node) structure)
        if "Filter" in m and "Group Key" in m:
            visited.append({"operator":"Filter", "id":node_id, "parent":m["parent"]})
            node_id += 1
            visited.append({"operator":"GroupBy", "id":node_id, "parent":node_id-1})
            node_id += 1
            p_node = {"operator":m['Node Type'], "id":node_id, "parent":node_id-1}
            visited.append(p_node)
        
        elif "Filter" in m:
                #p_node['extra_node'] = "Filter"
                visited.append({"operator":"Filter", "id":node_id, "parent":m["parent"]})
                node_id += 1 
                p_node = {"operator":m['Node Type'], "id":node_id, "parent":node_id-1}
                visited.append(p_node)
        
        elif "Group Key" in m:
                #p_node['extra_node'] = "GroupBy"
                visited.append({"operator":"GroupBy", "id":node_id, "parent":m["parent"]})
                node_id += 1
                p_node = {"operator":m['Node Type'], "id":node_id, "parent":node_id-1}
                visited.append(p_node)
        else:
            p_node = {"operator":m['Node Type'], "id":node_id, "parent":m["parent"]}
            visited.append(p_node)
                
        if "Plans" in m:
            for child in m['Plans']:
                child['parent'] = node_id
                queue.append(child)
        node_id += 1
    return visited
    #print(*visited, sep="\n")
    
#Flipping node list via converting child nodes to predecessors
#This structure is essential for DataFarm Unified Json Input
def flip_bfs(exec_plan):
    #create predecessors list
    for node in exec_plan:
        node['predecessors'] = []
    for node in exec_plan:
        if node.get("parent") is not None:
            parent = next(item for item in exec_plan if item["id"] == node.get("parent"))
            parent['predecessors'].append(node['id'])
            del node['parent']
        else:
            del node['parent']
    #append DataSink
    exec_plan.insert(0,{"operator":"Data Sink", "id":0, "predecessors":[1]})
    return exec_plan
    #print(*exec_plan, sep="\n")
    
#Completing unified json and saving
def complete_and_save(exec_plan_flipped):
    for node in exec_plan_flipped:
        if not node['predecessors']:
            node["type"] = "Data Source"
        elif node['id'] == 0:
            node["type"] = "sink"
        else:
            node["type"] = "operator"
        
        for count, value in enumerate(node['predecessors']):
            node['predecessors'][count] = {"id" : value}
        
        if not node['predecessors']:
            del node['predecessors']
    unified_json = {"executionPlan" : {"nodes" : exec_plan_flipped}}
    return unified_json
            
    

In [3]:
directory = 'sqls'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        sql = open(f, "r")
        cur = conn.cursor()
        cur.execute("EXPLAIN (FORMAT JSON)" + sql.read())
        plan = cur.fetchone()[0][0].get("Plan")
        print("Query ", f)
        
        unified_json = complete_and_save(flip_bfs(bfs(plan, 1)))
        
        with open('TPC_H_' + filename + 'Unified.json', 'w', encoding='utf-8') as nf:
            json.dump(unified_json, nf, ensure_ascii=False, indent=2)
        

Query  sqls/q11
Query  sqls/q3
Query  sqls/q1
Query  sqls/q13
Query  sqls/q17
Query  sqls/q21


In [4]:
cur.close()
conn.close()